In [ ]:
!pip install python-igraph
import pandas as pd
import igraph as ig

# Load node and edge data from CSV files
node_df = pd.read_csv('/content/longitudeandlatitude - longitudeandlatitude.csv')
edge_df = pd.read_csv('/content/monument_edges.csv')

# Create an empty graph
graph = ig.Graph(directed=False)

# Extract node IDs and labels
node_ids = node_df['ID'].tolist()
node_labels = node_df['Label'].tolist()

# Add nodes to the graph
graph.add_vertices(node_ids)
graph.vs['label'] = node_labels

# Extract and adjust edge pairs as tuples from the edge DataFrame
edge_pairs = list(zip(edge_df['Source'], edge_df['Target']))
adjusted_edge_pairs = [(source - 1, target - 1) for source, target in edge_pairs]

# Add edges to the graph with adjusted node IDs
graph.add_edges(adjusted_edge_pairs)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 11.4 MB/s eta 0:00:00


In [ ]:
# Add edge weights as a graph attribute (assuming 'Weight' column in edge_df)
graph.es['weight'] = edge_df['Weight'].tolist()

degree_centrality = graph.strength(weights='weight')

# Calculate betweenness centrality (considering edge weights, weighted)
betweenness_centrality = graph.betweenness(weights='weight')

# Calculate closeness centrality (considering edge weights, weighted)
closeness_centrality = graph.closeness(weights='weight')


# Calculate closeness centrality (considering edge weights, weighted)
closeness_centrality = graph.closeness(weights='weight')

eigen_centrality = graph.evcent(weights='weight')

# Create a dictionary to store the centrality measures for each node
centrality_measures = {
    'Degree Centrality': degree_centrality,
    'Betweenness Centrality': betweenness_centrality,
    'Closeness Centrality': closeness_centrality,
    'Eigenvalue Centrality': eigen_centrality,
}

# Function to get top 5 nodes based on centrality values
def get_top_nodes(centrality_values):
    sorted_nodes = sorted(range(len(centrality_values)), key=lambda k: centrality_values[k], reverse=True)
    return sorted_nodes[:5]

# Print top 5 nodes for each centrality measure
for name, values in centrality_measures.items():
    print(f"Top 5 nodes for {name}:")
    top_nodes = get_top_nodes(values)
    for node_index in top_nodes:
        node_label = node_labels[node_index]
        centrality_value = values[node_index]
        print(f"Node Label: {node_label}, Centrality Value: {centrality_value}")
    print()

Top 5 nodes for Degree Centrality:
Node Label: Humayun's Tomb, Centrality Value: 191.14
Node Label: Hazrat Nizamuddin Shrine, Centrality Value: 172.21
Node Label: Safdarjung Tomb, Centrality Value: 171.06000000000003
Node Label: sri Sri Radha Parthasarathi - ISKCON temple Sant Nagar, Centrality Value: 169.46000000000006
Node Label: Jhandewalan Mandir, Centrality Value: 168.79

Top 5 nodes for Betweenness Centrality:
Node Label: National Gallery of Modern Art, Centrality Value: 175.0
Node Label: Gurudwara Bangla Sahib, Centrality Value: 163.5
Node Label: Pragati Maidan, Centrality Value: 155.5
Node Label: Jhandewalan Mandir, Centrality Value: 126.0
Node Label: Jantar Mantar, Centrality Value: 97.0

Top 5 nodes for Closeness Centrality:
Node Label: Pragati Maidan, Centrality Value: 0.2387647901522789
Node Label: National Gallery of Modern Art, Centrality Value: 0.23625767837454717
Node Label: National Museum, Centrality Value: 0.2325942006512638
Node Label: National Science Centre Museum

In [ ]:
# Calculate the minimum and maximum degree centrality values
min_degree_centrality = min(degree_centrality)
max_degree_centrality = max(degree_centrality)

# Normalize the degree centrality values using Min-Max normalization
normalized_degree_centrality = [(value - min_degree_centrality) / (max_degree_centrality - min_degree_centrality) for value in degree_centrality]
max_betweenness_centrality = max(betweenness_centrality)

# Normalize the betweenness centrality values by dividing each value by the maximum
normalized_betweenness_centrality = [value / max_betweenness_centrality for value in betweenness_centrality]

# Create a dictionary to store the centrality measures for each node (including the normalized degree centrality and betweenness centrality)

centrality_data = pd.DataFrame({
    'Node ID': node_ids,
    'Node Label': node_labels,
   'Degree Centrality': normalized_degree_centrality,
    'Betweenness Centrality': normalized_betweenness_centrality,
    'Closeness Centrality': closeness_centrality,
    'Eigenvalue Centrality': eigen_centrality,
})

output_filename = 'centralities_output.csv'
centrality_data.to_csv(output_filename, index=False)

print("Centrality data saved to:", output_filename)

Centrality data saved to: centralities_output.csv


In [ ]:
ranks_df = pd.read_csv('/content/centrality_output (2).csv')

top_nodes_dict = {}

# List to store all nodes that appear in the top 5 ranks for all centralities
top_nodes_all_centralities = []

# Centrality measures column names (modify these based on your CSV file)
centrality_columns = ['Degree_Rank', 'Betweeness_Rank', 'closeness_rank', 'Eigenvalue_rank']

# Find top 5 nodes for each centrality measure
for centrality_measure in centrality_columns:
    top_nodes = ranks_df.nsmallest(10, centrality_measure)['Node ID'].tolist()
    top_nodes_dict[centrality_measure] = top_nodes
    top_nodes_all_centralities.extend(top_nodes)

# Find the nodes that appear in the top 5 ranks for all centralities
top_nodes_all_centralities = list(set(top_nodes_all_centralities))

# Print top 5 nodes for each centrality measure
for centrality_measure, top_nodes in top_nodes_dict.items():
    print(f"Top 10 nodes for {centrality_measure}:")
    for node_id in top_nodes:
        node_label = ranks_df.loc[ranks_df['Node ID'] == node_id, 'Node Label'].values[0]
        centrality_rank = ranks_df.loc[ranks_df['Node ID'] == node_id, centrality_measure].values[0]
        print(f"Node ID: {node_id}, Node Label: {node_label}, Rank: {centrality_rank}")
    print()

# Print nodes that appear in the top 5 ranks for all centralities
print("Nodes with top ranks in all centralities:")
for node_id in top_nodes_all_centralities:
    node_label = ranks_df.loc[ranks_df['Node ID'] == node_id, 'Node Label'].values[0]
    print(f"Node ID: {node_id}, Node Label: {node_label}")

Top 10 nodes for Degree_Rank:
Node ID: 13, Node Label: Humayun's Tomb, Rank: 1
Node ID: 12, Node Label: Hazrat Nizamuddin Shrine, Rank: 2
Node ID: 39, Node Label: Safdarjung Tomb, Rank: 3
Node ID: 16, Node Label: sri Sri Radha Parthasarathi - ISKCON temple Sant Nagar, Rank: 4
Node ID: 19, Node Label: Jhandewalan Mandir, Rank: 5
Node ID: 37, Node Label: Raj Ghat, Rank: 6
Node ID: 36, Node Label: Rahim-Khane-Khanam Tomb, Rank: 7
Node ID: 25, Node Label: Amrit Udyan, Rank: 8
Node ID: 29, Node Label: National Rail Museum, Rank: 9
Node ID: 23, Node Label: Lodhi Tomb, Rank: 10

Top 10 nodes for Betweeness_Rank:
Node ID: 26, Node Label: National Gallery of Modern Art, Rank: 1
Node ID: 11, Node Label: Gurudwara Bangla Sahib, Rank: 2
Node ID: 33, Node Label: Pragati Maidan, Rank: 3
Node ID: 19, Node Label: Jhandewalan Mandir, Rank: 4
Node ID: 18, Node Label: Jantar Mantar, Rank: 5
Node ID: 31, Node Label: Nehru Planetarium, Rank: 6
Node ID: 46, Node Label: Bharat Darshan Park, Rank: 7
Node ID: 

In [ ]:
input_file = "/content/centrality_output (2).csv"
df = pd.read_csv(input_file)

# Calculate the average rank for each row (node)
df["AverageRank"] = df[["Degree_Rank", "Betweeness_Rank", "closeness_rank", "Eigenvalue_rank"]].mean(axis=1)

# Save the updated DataFrame to a new CSV file
output_file = "output_ranks_computed.csv"
df.to_csv(output_file, index=False)

print("New CSV file generated with the average rank column.")

New CSV file generated with the average rank column.
